# Deploying Puppet Locally

## 1. Intro to Module 2: Deploying Puppet

Welcome back, congrats on making it through the first lab of the course. In the last module, we looked into some basic configuration management concepts, and what those look like in Puppet. In this module, we're going to dive deeper into how to use these tools. We'll kick off with how to install Puppet on your computer and how to use a simple test setup to check your rules work as expected. This test setup will let you try out the examples in these videos on your own computer. 

We'll also show you how to configure the typical client-server set-up with Puppet clients connecting and authenticating to the Puppet server to get the rules that they should apply and on top of this, we'll talk about how to use testing techniques and releasing best practices to safely deploy changes to clients of our configuration management system. 

You've already learned the basics of Puppet resources. We've seen the three most important resources, packages, files, and services. In this module, we'll check out more ways of using the basic resources and Puppet's DSL. We'll look into how you can apply different sets of rules to different nodes in your fleet, how you can organize your rules so that they're easier to maintain, and a bunch of other Puppet best-practices. We'll wrap up with another lab exercise. This time though, you'll be flexing your Puppet skills to manage the deployment and pulling the strings to make the system do what you want. Ready? Let's dive in.

## 2. Applying Rules Locally

Up to now we've been getting to know Puppet syntax and understanding the different resources available. It's now time for the next step, trying out some Puppet rules on our local computer. In an earlier video, we called out that Puppet is usually deployed in a client-server architecture. But that's not the only way we can use Puppet. We can also use it as a stand-alone application run from the command line. This is common when testing new configurations. It can be the preferred configuration for complex setups where connecting to a master is no longer the best approach. 

When using a stand-alone Puppet, the same computer processes the facts, calculates the rules that need to be applied, and makes any necessary changes locally. So to get started with our Puppet deployment, let's first install Puppet and then we can start experimenting with running rules locally. In later videos, we'll check out how to create a client-server deployments. 

As we've called out, Puppet is available on a number of different platforms. We can either install it from the package management system available in the OS or download it from the official website. Both options work fine and the best one to choose will depend on our specific needs. For this exercise, we'll just go with the Puppet packages provided by the Ubuntu distribution. We'll do that by installing the Puppet master package using sudo apt install puppet-master.

### 2.1 Example

We now have the package installed and can start trying out a few rules. We'll begin by creating the simplest possible Puppet file. We can make it more complex as we improve our deployments. For this example, we want to use Puppet to make sure that some useful tools for debugging problems are installed on each computer in our fleet. To do this, we first have to create a file where we'll store the rules that we want to apply. In Puppet lingo, these files are called **manifests** and they must end with a.pp extension. So we'll create a new file called tools.pp and in this file, we'll create a package resource. We'll start by managing the htop package which is a tool similar to top that can show us some extra information. We'll state that we want Puppet to ensure that we have this package present on our computer. 

```
package { 'htop':
  ensure => present,
}
```
Cool. That was simple. That's all we have to do. This resource will take care of installing the package for us. Let's save the file and try it out. But before actually applying the rules, we want to check that the command isn't present yet.

```
buraiyen@ubby:~/Documents/Google-IT/5-CMC/Week-2$ sudo puppet apply -v tools.pp
Notice: Compiled catalog for ubby.home in environment production in 0.19 seconds
Info: Applying configuration version '1600368814'
Notice: /Stage[main]/Main/Package[htop]/ensure: ensure changed 'purged' to 'present'
Info: Creating state file /var/lib/puppet/state/state.yaml
Notice: Finished catalog run in 7.42 seconds
buraiyen@ubby:~/Documents/Google-IT/5-CMC/Week-2$ 
```

The -v flag tells Puppet that we want to get verbose output which will tell us what's going on while Puppet is applying the rules in the file that we pass to it. So here, Puppet first told us that it was loading the facts. Then, that it compiled a catalog. After that, it told us that it was applying the current configuration. Then, that it installed the package we requested. Finally, it let us know that it finished applying this catalog. 

### 2.2 Catalogs

You're probably wondering, what's a catalog? We called out in an earlier video that after loading all facts for a computer, the server calculates which rules actually need to be applied. For example, if a packet should only be installed when a certain condition is met, this condition is evaluated on the server side based on the gathered facts. The **catalog** *is the list of rules that are generated for one specific computer once the server has evaluated all variables, conditionals, and functions.* In this example, the catalog will be exactly the same as our code because the code didn't include any variables, functions, or conditionals. More complex sets of rules can lead to different catalogs depending on fact values. It's now time to check if our rules actually works. Let's try running the htop command again now that Puppet has installed it for us.

![img1](https://github.com/Brian-E-Nguyen/Google-IT-Automation-with-Python/blob/5-Config-Management-and-Cloud/5-Config-Management-and-Cloud/Week-2-Deploying-Puppet/img/img1.jpg?raw=true)

Yes, this time it worked. If our computer was misbehaving, we could now use this tool to get a better idea why. But fortunately, our computer's on its best behavior. So we'll exit now using q. Let's see what happens if we try to apply the Puppet rules again now that the package is installed.

Puppet's smart. It noticed that the package is already installed so it didn't try to install the package again. This means it applied the catalog much faster because nothing had to be changed. We've now seen how to write a Puppet resource in a manifest file and then use puppet apply to apply those rules to one computer. Up next, we'll check out how we can manage relationships between different Puppet resources and what that looks like when applied.

## 3. Managing Resource Relationships

In our last video, we wrote a very simple manifest which we then applied locally. That was a great way to practice applying Puppet rules, but it was super-simple. Let's challenge ourselves with something a bit more tricky. The Puppet manifests that we use to manage computers in our fleet usually include a bunch of different resources that are related to each other. You're not going to configure a package that's not installed and you don't want to start a service until both the package and the configuration are in place. Puppets lets us control this with resource relationships. Let's check this out in an example. We have a file called ntp.pp, that has a bunch of resources related to the NUTS configuration like the one we've seen in an earlier video.

```pp
class ntp {
  package { 'ntp':
    ensure => latest,
  }
  file { '/etc/ntp.conf':
    source  => '/home/user/ntp.conf',
    replace => true
    require => Package['ntp'],
    notify  => Service['ntp'],
  }
  service { 'ntp':
    enable => true,
    ensure => running,
    require => File['/etc/ntp.conf'],	
  }
}
include ntp	
```

This time, on top of declaring the resources that we need to manage, we're also declaring a few relationships between them. We see that the configuration file requires the NTP package and the service requires the configuration file. This way, Puppet knows that before starting the service, the configuration file needs to be correctly set, and before sending the configuration file, the package needs to be installed. 

We're also declaring that the NTP service should be notified if the configuration file changes. That way, if we make additional changes to the contents of the configuration file in the future, the service will get reloaded with the new settings. 

If you look closely, you might notice that the resource types are written in lowercase, but relationships like require or notify use uppercase for the first letter of the resource. This is part of Puppet syntax. **We write resource types in lowercase when declaring them, but capitalize them when referring to them from another resource's attributes.** 

This sounds confusing right now, don't worry. It might take a while to wrap your head around it, but it will eventually click. Now, one last thing. At the bottom of the file, we have a call to include NTP. That's why we told Puppet that we want to apply the rules described in a class. For this example, we put the definition of the class and the call to include the class in the same file. Typically, the class is defined in one file and include it in another. We'll checkout examples for this in later videos. All right. Let's apply these rules locally.

![img2](https://github.com/Brian-E-Nguyen/Google-IT-Automation-with-Python/blob/5-Config-Management-and-Cloud/5-Config-Management-and-Cloud/Week-2-Deploying-Puppet/img/img2.jpg?raw=true)

Great. Our rules have run and in the verbose output, we can see that it did a bunch of things. First, it installed the package, then it checked that the configuration file needed to be updated and so it changed its contents. Finally, after changing the contents of the configuration, Puppet knew to restart the NTP service. We see here how our Puppet rules have translated into a few different actions. That's cool, but it's about to get even better. Let's make a change to the configuration file by editing the ntp.com file in this directory.

![img3](https://github.com/Brian-E-Nguyen/Google-IT-Automation-with-Python/blob/5-Config-Management-and-Cloud/5-Config-Management-and-Cloud/Week-2-Deploying-Puppet/img/img3.jpg?raw=true)

This is the configuration values by the NTP service. It's currently using a bunch of servers from ntp.org. But instead of those servers, we want to try out the NTP servers provided by Google. These are called time1.google.com, and then time2, time3, and time4.

![img4](https://github.com/Brian-E-Nguyen/Google-IT-Automation-with-Python/blob/5-Config-Management-and-Cloud/5-Config-Management-and-Cloud/Week-2-Deploying-Puppet/img/img4.jpg?raw=true)

We've made the change, we'll save with :WQ and then rerun our Puppet rules with the new configuration file.

![img5](https://github.com/Brian-E-Nguyen/Google-IT-Automation-with-Python/blob/5-Config-Management-and-Cloud/5-Config-Management-and-Cloud/Week-2-Deploying-Puppet/img/img5.jpg?raw=true)

Awesome. Puppet updated the configuration file with the new contents and then refresh the service, so it loaded the config. Success. In this video, we've seen how we can apply a Puppet manifests that includes a class with a bunch of resources. We grouped all of the information related to the NTP service in a manifest specific to it, which is common practice when dealing with Puppet rules. We want to keep related operations together and separate things that are unrelated. Up next, we'll look into how we can do that using Puppet modules.

## 4. Organizing Your Puppet Modules

In any configuration management deployment, there's usually a lot of different things to manage. We might want to install some packages, copy some configuration files, start some services, schedule some periodic tasks, make sure some users and groups are created and have access to specific devices, and maybe execute a few commands that aren't provided by existing resources. 

On top of that, there might be different configurations applied to the different computers in the fleet. For example, workstations and laptops might include resources that aren't used on servers. Each distinct type of server will need its own specific setup. There's a lot of different things to manage. We need to organize all these resources and information in a way that helps us maintain them long-term. This means grouping related resources, giving the groups good names, and making sure that the organization will make sense to new users. 

### 4.1 Info on Modules

In puppet, we organize our manifests into modules. A **module** *is a collection of manifests and associated data.* We can put any resource we want into a module, but to keep our configuration management organized, we'll group things together under a sensible topic. For example, we could have a module for everything related to monitoring the computer's health, another one for setting up the network stack, and yet another one for configuring a web serving application. 

So the module ship the manifest in the associated data, but how is this organized? All manifests gets stored in a directory called manifests. The rest of the data is stored in different directories depending on what it does. The files directory includes files that are copied into the client machines without any changes, like the ntp.conf file that we saw in our last video. The template's directory includes files that are preprocessed before they've been copied into the client machines.

![img6](https://github.com/Brian-E-Nguyen/Google-IT-Automation-with-Python/blob/5-Config-Management-and-Cloud/5-Config-Management-and-Cloud/Week-2-Deploying-Puppet/img/img6.jpg?raw=true)

### 4.2 Example

These templates can include values that get replaced after calculating the manifests, or sections that are only present if certain conditions are valid. There's a bunch more directories that can be part of a module depending on what exactly the module does. But you don't need to worry about these when creating your first puppet module. You can start with the simple module that just has one manifest in the Manifest directory. This file should be called `init.pp` and it should define a class with the same name as the module that you're creating. Then any files that your rules use need to be stored in the files or templates directories depending on whether you copy them directly or need to preprocess them. For example, this is how the NTP class that we saw in our last video looks like when turned into a module.

![img7](https://github.com/Brian-E-Nguyen/Google-IT-Automation-with-Python/blob/5-Config-Management-and-Cloud/5-Config-Management-and-Cloud/Week-2-Deploying-Puppet/img/img7.jpg?raw=true)

There's an init.pp file, which contains the NTP classes that we saw before, and the ntp.conf file that gets deployed onto the machine is now stored in the files directory. Modules like these can look pretty much the same no matter who's using them. That's why over time, system administrators using puppet have shared the modules they've written, letting others use the same rules. By now, there's a large collection of prepackaged modules that are shipped and ready to use. If one of those modules does what we want, we can just install it on our Puppet server and use it in our deployments. Let's install the Apache module provided by Puppet Labs to check out how this works.

```
buraiyen@ubby:~$ sudo apt install puppet-module-puppetlabs-apache
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following extra packages will be installed:
  puppet-module-puppetlabs-concat puppet-module-puppetlabs-stdlib
The following NEW packages will be installed:
  puppet-module-puppetlabs-apache puppet-module-puppetlabs-concat
  puppet-module-puppetlabs-stdlib
0 upgraded, 3 newly installed, 0 to remove and 61 not upgraded.
Need to get 114 kB of archives.
After this operation, 574 kB of additional disk space will be used.
Do you want to continue? [Y/n] Y
Get:1 http://us.archive.ubuntu.com/ubuntu/ trusty/universe puppet-module-puppetlabs-stdlib all 4.1.0-3 [43.6 kB]
Get:2 http://us.archive.ubuntu.com/ubuntu/ trusty/universe puppet-module-puppetlabs-concat all 1.0.0-2 [9,914 B]
Get:3 http://us.archive.ubuntu.com/ubuntu/ trusty/universe puppet-module-puppetlabs-apache all 0.10.0-3 [60.7 kB]
Fetched 114 kB in 0s (173 kB/s)                       
Selecting previously unselected package puppet-module-puppetlabs-stdlib.
(Reading database ... 170784 files and directories currently installed.)
Preparing to unpack .../puppet-module-puppetlabs-stdlib_4.1.0-3_all.deb ...
Unpacking puppet-module-puppetlabs-stdlib (4.1.0-3) ...
Selecting previously unselected package puppet-module-puppetlabs-concat.
Preparing to unpack .../puppet-module-puppetlabs-concat_1.0.0-2_all.deb ...
Unpacking puppet-module-puppetlabs-concat (1.0.0-2) ...
Selecting previously unselected package puppet-module-puppetlabs-apache.
Preparing to unpack .../puppet-module-puppetlabs-apache_0.10.0-3_all.deb ...
Unpacking puppet-module-puppetlabs-apache (0.10.0-3) ...
Setting up puppet-module-puppetlabs-stdlib (4.1.0-3) ...
update-alternatives: using /usr/share/puppet/modules.available/puppetlabs-stdlib to provide /usr/share/puppet/modules/stdlib (puppet-module-stdlib) in auto mode
Setting up puppet-module-puppetlabs-concat (1.0.0-2) ...
update-alternatives: using /usr/share/puppet/modules.available/puppetlabs-concat to provide /usr/share/puppet/modules/concat (puppet-module-concat) in auto mode
Setting up puppet-module-puppetlabs-apache (0.10.0-3) ...
update-alternatives: using /usr/share/puppet/modules.available/puppetlabs-apache to provide /usr/share/puppet/modules/apache (puppet-module-apache) in auto mode
```

We've installed the module. Let's have a quick look at its contents. First, we'll change into the directory where the module files are stored and list its contents.

```
buraiyen@ubby:/usr$ cd share/puppet/modules.available/puppetlabs-apache/
buraiyen@ubby:/usr/share/puppet/modules.available/puppetlabs-apache$ ls -l
total 40
drwxr-xr-x 2 root root  4096 Sep 18 11:25 files
drwxr-xr-x 3 root root  4096 Sep 18 11:25 lib
drwxr-xr-x 7 root root  4096 Sep 18 11:25 manifests
-rw-r--r-- 1 root root 16108 Dec  5  2013 metadata.json
-rw-r--r-- 1 root root   402 Dec  5  2013 Modulefile
drwxr-xr-x 5 root root  4096 Sep 18 11:25 templates
drwxr-xr-x 2 root root  4096 Sep 18 11:25 tests
buraiyen@ubby:/usr/share/puppet/modules.available/puppetlabs-apache$ 
```

We see the files, manifests, and templates directories that we mentioned. On top of that, there's a lib directory that adds functions and fact to the ones already shipped by puppet. The metadata.json file includes some additional data about the module we just installed, like which versions of which operating systems it's compatible with. Let's peek into the manifest directory.

```
buraiyen@ubby:/usr/share/puppet/modules.available/puppetlabs-apache$ ls -l manifests/
total 112
-rw-r--r-- 1 root root  1722 Dec  5  2013 balancermember.pp
-rw-r--r-- 1 root root  2481 Dec  5  2013 balancer.pp
drwxr-xr-x 2 root root  4096 Sep 18 11:25 confd
-rw-r--r-- 1 root root   235 Dec  5  2013 default_confd_files.pp
drwxr-xr-x 2 root root  4096 Sep 18 11:25 default_mods
-rw-r--r-- 1 root root  3295 Dec  5  2013 default_mods.pp
-rw-r--r-- 1 root root   353 Dec  5  2013 dev.pp
-rw-r--r-- 1 root root  9811 Dec  5  2013 init.pp
-rw-r--r-- 1 root root   262 Dec  5  2013 listen.pp
drwxr-xr-x 2 root root  4096 Sep 18 11:25 mod
-rw-r--r-- 1 root root  3339 Dec  5  2013 mod.pp
-rw-r--r-- 1 root root   268 Dec  5  2013 namevirtualhost.pp
-rw-r--r-- 1 root root  1445 Dec  5  2013 package.pp
-rw-r--r-- 1 root root  7977 Dec  5  2013 params.pp
drwxr-xr-x 2 root root  4096 Sep 18 11:25 peruser
-rw-r--r-- 1 root root   290 Dec  5  2013 php.pp
-rw-r--r-- 1 root root   280 Dec  5  2013 proxy.pp
-rw-r--r-- 1 root root   311 Dec  5  2013 python.pp
-rw-r--r-- 1 root root   734 Dec  5  2013 service.pp
-rw-r--r-- 1 root root   343 Dec  5  2013 ssl.pp
drwxr-xr-x 2 root root  4096 Sep 18 11:25 vhost
-rw-r--r-- 1 root root 15371 Dec  5  2013 vhost.pp
buraiyen@ubby:/usr/share/puppet/modules.available/puppetlabs-apache$ 
```

That's a lot of files, like how we split the different things that we want to manage into separate modules. We can also split each separate functionality that we want to configure into separate manifests. This helps us organize our code when we make changes to it, and to see how this directory also contains its own init.pp. As we called out, this manifest is special. It needs to always be present because it's the first one that's read by puppet when a module gets included. So how do we include a module like this one? It's pretty easy. Let's create a manifest file that includes the module we've just installed.

```pp
include ::apache
```

Here, we're telling Puppet to include the Apache module. The double colon before the module name, let's puppet know that this is a global module. Let's save this file now and apply it using Puppet apply like we did before.

Our manifest was super-simple, it just include the Apache module. But by including the module, we got puppet to apply all the rules run by default in the module. We now have an Apache server configured and ready to run on this machine. We've just seen how we can organize our code in modules and how we can even use modules provided by other teams so we don't have to reinvent the wheel. Up next, there's a reading with pointers to more information, followed by a quick quiz. After that, meet me over in the next video, where we'll check out what we need to do to deploy our rules to more machines.

## 5. More Information About Deploying Puppet Locally

Check out the following links for more information:

- https://puppet.com/docs/puppet/latest/style_guide.html
- https://puppet.com/docs/puppetserver/latest/install_from_packages.html